In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
import joblib
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [2]:
df= pd.read_csv(r"C:\Users\keyke\Documents\botcam\Proyecto\scr\Data\Processed\datafims.csv")


In [6]:
import joblib

# Carga el modelo entrenado
model = joblib.load("model/xgb_film_model.pkl")

# Obtiene las características utilizadas por el modelo
feature_names = model.get_booster().feature_names

# Imprime la lista de características
print(feature_names)
print(type(feature_names[0]))

['rating', 'genre', 'year', 'votes', 'budget', 'runtime']
<class 'str'>


In [ ]:
from flask import Flask, render_template, request
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb

app = Flask(__name__)
model = pickle.load(open("model/xgb_film_model.pkl", "rb"))

# Mapeo de categorías de calificación por edades y género a números
calificacion_mapper = {'R': 1, 'PG': 2, 'G': 3, 'Not Rated': 4, 'NC-17': 5, 'Approved': 6, 'PG-13': 7, 'Unrated': 8, 'X': 9, 'TV-PG': 10, 'TV-MA': 11, 'TV-14': 12}
genero_mapper = {'Drama': 1, 'Adventure': 2, 'Action': 3, 'Comedy': 4, 'Horror': 5, 'Biography': 6, 'Crime': 7, 'Fantasy': 8, 'Family': 9, 'Animation': 10, 'Romance': 11, 'Music': 12, 'Western': 13, 'Thriller': 14, 'Sci-Fi': 15, 'Mystery': 16, 'Sport': 17, 'Musical': 18}

@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        # Obtener los valores ingresados desde el formulario
        rating_text = request.form.get("rating")
        genre_text = request.form.get("genre")
        year = int(request.form.get("year"))
        votes = int(request.form.get("votes"))
        budget = int(request.form.get("budget"))
        runtime = int(request.form.get("runtime"))

        print("Valores ingresados:")
        print("Rating:", rating_text)
        print("Genre:", genre_text)
        print("Year:", year)
        print("Votes:", votes)
        print("Budget:", budget)
        print("Runtime:", runtime)

        # Convertir calificación por edades y género a números usando el mapeo
        rating = calificacion_mapper.get(rating_text, 0)
        genre = genero_mapper.get(genre_text, 0)

        # Crear un DataFrame con las características
        data = pd.DataFrame([[rating, genre, year, votes, budget, runtime]], columns=['rating', 'genre', 'year', 'votes', 'budget', 'runtime'])

        # Realizar la predicción usando el modelo
        prediction = model.predict(data)[0]

        return render_template("index.html", prediction_text="El Ingreso Bruto de nuestra película es: {}".format(prediction))

    return render_template("index.html") 

if __name__ == "__main__":
    app.run(debug=True)

